In [56]:
import glob
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

In [7]:
files = glob.glob('./afp_eng_*')

In [65]:
file = files[0]
with open(file) as f:
    soup = BeautifulSoup(f.read(), "html.parser")

docs = soup.select('doc')
print(file, len(docs))

for doc in docs[:10]:
    head = doc.select('headline')
    paras = doc.select('text p')
    if len(head) == 0 or len(paras) == 0:
        continue
    assert len(head) == 1, 'multiple headlines'
    print('HEADLINE:\t', head[0].get_text().strip())
    print('TEXT         :\t', paras[0].get_text().replace('\n', ' ').strip())
    print(len(paras))

## doc.select('text p')がないものもあるが，これはtextタグのみで囲われており，データの羅列のようなものが多いっぽいので，
## 要約として使用する分にはデータに含める必要はなさそう

./afp_eng_200808 22826
HEADLINE:	 Football: Manchester United's new dawn in jeopardy
TEXT         :	 A new dawn looked likely for Manchester United after Alex Ferguson's team won the European Champions League in Moscow in May to complete a domestic and European double.
18
HEADLINE:	 Football: Scolari's sunshine summit gives Chelsea winning feeling
TEXT         :	 As Luiz Felipe Scolari stepped ashore from Roman Abramovich's Pelorus yacht the Brazilian was already convinced he had made the right decision to join Chelsea.
19
HEADLINE:	 Football: French football icon Thuram retires
TEXT         :	 Lilian Thuram's retirement has meant one of France's finest players of the past decade, who transcended the sport and made an impact in political and social circles, has finally hung up his boots.
19
HEADLINE:	 Dollar gains on better-than-feared US job report
TEXT         :	 The dollar firmed against the euro Friday after the US government reported fewer than expected job losses in July and fres

In [49]:
list(map(lambda x: x.get_text().strip(), head))

['Tennis: Djokovic schools Gulbis for second time this season']

In [ ]:
with open('./document.txt', 'w') as dt, \
         open('./headline.txt', 'w') as ht, \
         open('./document1.txt', 'w') as dt1, \
         open('./document3.txt', 'w') as dt3:
    for filenum, file in enumerate(files, 1):
        print(f'{filenum}/{len(files)}: {file}')
        with open(file) as sf:
            soup = BeautifulSoup(sf.read(), 'html.parser')
            docs = soup.select('doc')
            for doc in tqdm(docs):
                head = doc.select('headline')
                paras = doc.select('text p')
                assert len(head) <= 1, 'multiple headlines'
                if len(head) == 0 or len(paras) == 0:
                    continue
                ht.write(head[0].get_text().replace('\n', ' ').strip() + '\n')
                paras =  list(map(lambda x: x.get_text().replace('\n', ' ').strip(), paras))
                dt.write(' '.join(paras) + '\n')
                dt1.write(paras[0] + '\n')
                dt3.write(' '.join(paras[:3]) + '\n')

1/146: ./afp_eng_200808



2/146: ./afp_eng_200501



3/146: ./afp_eng_200908



4/146: ./afp_eng_200905



5/146: ./afp_eng_200310


...


55/146: ./afp_eng_200302



56/146: ./afp_eng_199405



57/146: ./afp_eng_200511



58/146: ./afp_eng_200902



59/146: ./afp_eng_199603


## データを分ける

In [2]:
! wc -l headline.txt

2196479 headline.txt


In [12]:
from os import makedirs
import numpy as np
import random

In [30]:
makedirs('doc1/org', exist_ok=True)
makedirs('doc3/org', exist_ok=True)

In [ ]:
source_docs = {'document1.txt': 'doc1', 'document3.txt': 'doc3'}
val_test_rate = 0.005
train_rate = 1 - 2 * val_test_rate
print(train_rate, val_test_rate, val_test_rate)
random_seed = 100
data_len = 2196479

def enclosure(key_file_dic):
    def writer(key, doc, head):
        docf, headf = key_file_dic[key]
        docf.write(doc.strip() + '\n')
        headf.write(head.strip() + '\n')
    return writer

for source_doc in source_docs:
    print(source_doc)
    with open(source_doc) as sif, \
         open('headline.txt') as sof:
        io_zipped = zip(sif.readlines(), sof.readlines()) # list化するとmemory error
        mlabels = np.zeros(data_len, dtype=int)
        mlabels[:int(2*val_test_rate*data_len)] = 1 # val label
        mlabels[int(val_test_rate*data_len):int((2*val_test_rate*data_len))] = 2 # test label
        random.shuffle(mode_labels)
        with open(f'{source_docs[source_doc]}/org/train.doc', 'w') as trd, \
             open(f'{source_docs[source_doc]}/org/train.head', 'w') as trh, \
             open(f'{source_docs[source_doc]}/org/val.doc', 'w') as vad, \
             open(f'{source_docs[source_doc]}/org/val.head', 'w') as vah, \
             open(f'{source_docs[source_doc]}/org/test.doc', 'w') as ted, \
             open(f'{source_docs[source_doc]}/org/test.head', 'w') as teh:
            key_file_dic = {'train': [trd, trh],
                            'val': [vad, vah],
                            'test': [ted, teh]}
            keys = ['train', 'val', 'test']
            writer = enclosure(key_file_dic)
            for i, ((doc, head), mlabel) in enumerate(zip(io_zipped, mlabels)):
                print(f'{i}/{data_len}', end='\r')
                writer(keys[mlabel], doc, head)

0.99 0.005 0.005
document1.txt
document3.txt79


In [2]:
! wc -l doc1/train.doc
! wc -l doc1/train.head
! wc -l doc1/val.head
! wc -l doc1/test.head
! wc -l doc3/train.doc

2174515 doc1/train.doc
2174515 doc1/train.head
10982 doc1/val.head
10982 doc1/test.head
2174515 doc3/train.doc


## sentencepieceでunigram encoding

In [9]:
import sentencepiece as spm
from tqdm import tqdm_notebook as tqdm

In [2]:
! mkdir -p doc1/sp_unigram
! mkdir -p doc3/sp_unigram

In [11]:
doc_type = 'doc1'  # 'doc3'
model_type = 'unigram'  # 'bpe'
if model_type == 'unigram':
    srcdir = f'org'
    sptrain_params = f'--input={doc_type}/{srcdir}/train.doc --model_prefix={doc_type}/sp_{model_type}/trained_model --vocab_size=16000 --model_type=unigram'
elif model_type == 'bpe':
    srcdir = 'mecab/juman'
    sptrain_params = f'--input={srcdir}/train.doc --model_prefix=sp_{model_type}/trained_model --vocab_size=16000 --model_type=bpe --split_by_whitespace=true'

In [ ]:
spm.SentencePieceTrainer.Train(sptrain_params)

In [14]:
modes = ['train', 'val', 'test']
cols = {'doc': 'document', 'head': 'headline'}
for mode in modes:
    for col in cols:
        print('SOURCE: ', f'{doc_type}/{srcdir}/{mode}.{col}')
        wc_res = !wc -l {doc_type}/{srcdir}/{mode}.{col}
        num_lines = int(wc_res[0].split()[0])
        with open(f'{doc_type}/{srcdir}/{mode}.{col}') as srcf, \
             open(f'{doc_type}/sp_{model_type}/{mode}.{col}', 'w') as txtf:
            for l in tqdm(srcf, total=num_lines):
                txtf.write(' '.join(sp.EncodeAsPieces(l.strip())) + '\n')

SOURCE:  doc1/org/train.doc



SOURCE:  doc1/org/train.head



SOURCE:  doc1/org/val.doc



SOURCE:  doc1/org/val.head



SOURCE:  doc1/org/test.doc



SOURCE:  doc1/org/test.head
